# Training Tests for Part Of Speech tagging

This notebook is dedicated to start working with the PoS dataset already pre-processed and the column networks that I'm creating.

The network will be constructed from small parts, each will be trained on top of the previous one, adding a new column and decoder.


In [1]:
from datetime import datetime
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

from langmodels.models import *
import langmodels.utf8codec as utf8codec
from langmodels.utils.tools import *
from langmodels.utils.preprocess_conllu import *



/home/leo/venv3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/leo/venv3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/leo/venv3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/leo/venv3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dty

Load the embeddings first

In [2]:
# load the codebook and all the dictionaries mapping the data
# utf8codes, txt2code, code2txt, txt2num, num2txt = utf8codec._load_codebook()
utf8codes = np.load("./utf8-codes/utf8_codebook_overfit_matrix_2seg_dim64.npy")

In [3]:
utf8codes = utf8codes.reshape(1987,64)

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
net = Conv1DPoS(utf8codes)
net = net.to(device)

NameError: name 'Conv1DPoS' is not defined

In [ ]:
count_parameters(net)

for the original network 11266152 parameters, I have cut the number of features and dimensions to make it smaller

for nlayers = 5 of dim 5 is 6912424 and 6846888 trainable

for the following Conv1DPartOfSpeech the number of parameters is: 2161960 where 2096424 are trainable

    nchannels_in=[64, 128, 256, 512, 256],
    nchannels_out=[128, 256, 512, 256, 96],
    kernels=[3, 3, 3, 3, 3],
    nlayers=[6, 6, 4, 4, 3],
    groups=[1, 4, 8, 4, 1],
    
And LinearUposDeprelDecoder params are:

    lin_in_dim=96, 
    lin_hidd_dim=768,
    upos_dim=18, 
    deprel_dim=278,

In [ ]:
count_trainable_parameters(net)

Datasets are the one that are heavy, so I'll just load them and check what happens

In [ ]:
dataset_train = "/home/leo/projects/Datasets/text/UniversalDependencies/ud-treebanks-v2.4/traindev_np_batches_779000x3x1024_uint16.npy"

In [ ]:
data_train = np.load(dataset_train)

In [ ]:
data_train.shape

In [ ]:
len(data_train)

In [ ]:
data_train.dtype

In [ ]:
dta_train_txt = data_train[:,0,:]
dta_train_upos = data_train[:,1,:]
dta_train_deprel = data_train[:,2,:]

In [ ]:
# x = torch.from_numpy(dta_train_txt[:50].astype("int64")).to(device)

In [ ]:
# txtcode, positions, latent, dec = net(x)
# last_latent = latent[-1]
# upos, deprel = dec

In [ ]:
# txtcode.shape, positions.shape, last_latent.shape, upos.shape, #  deprel.shape

In [ ]:
# out = torch.cat([upos,deprel], dim=2)

In [ ]:
# out.shape

In [ ]:
# upos and deprel data are given by indices, this keeps memory as low as possible, but they need to be encoded
upos_eye = torch.eye(len(UPOS))
deprel_eye = torch.eye(len(DEPREL))
with torch.no_grad():
    upos_emb = nn.Embedding(*upos_eye.shape)
    upos_emb.weight.data.copy_(upos_eye)
    upos_emb = upos_emb.to(device)

    deprel_emb = nn.Embedding(*deprel_eye.shape)
    deprel_emb.weight.data.copy_(deprel_eye)
    deprel_emb.to(device)


In [ ]:

# from https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(data, n, dim=0):
    """Yield successive n-sized chunks from data by the dimension dim"""
    for i in range(0, data.shape[dim], n):
        yield data[i:i + n,:,:]


In [ ]:
def loss_function(upos, deprel, target_upos, target_deprel):

    # TODO check a more sofisticated loss function, for the moment only the sum to see if it runs
    # the issue is that upos is easier than deprel (18 vs 278 classes)
#     upos_loss = F.mse_loss(upos, target_upos)
#     deprel_loss = F.mse_loss(deprel, target_deprel)
    # issue with the size of target and tensors for cross_entropy ... I don't understand
#     upos_loss = F.cross_entropy(upos, target_upos)
#     deprel_loss = F.cross_entropy(deprel, target_deprel)
#     print(upos.shape, target_upos.shape, deprel.shape, target_deprel.shape)
    upos_loss = F.nll_loss(upos, target_upos)
    deprel_loss = F.nll_loss(deprel, target_deprel)
#     upos_loss = F.kl_div(upos, target_upos)
#     deprel_loss = F.kl_div(deprel, target_deprel)
    loss = upos_loss + deprel_loss
#     loss = F.kl_div(torch.cat([upos, deprel], dim=-1).contiguous(), torch.cat([target_upos, target_deprel], dim=-1).contiguous())
    return loss


In [ ]:
writer = SummaryWriter()

In [ ]:
# indata = torch.from_numpy(data_train[-2:,0,:].astype("int64")).to(device)

In [ ]:
# indata.shape

In [ ]:
# %%time
# testing tensorboard add_graph to see if the network graph is drawn correctly ;)
# indata = torch.from_numpy(data_train[-2:,0,:].astype("int64")).to(device)
# writer.add_graph(net, indata)
# Kernel dies when I do this ... so ... :O

In [ ]:
def train(model, optimizer, loss_function, batches, epoch, ndatapoints, device, log_interval=100):
    model.train()
    train_loss = 0
#     batch_loss = []
    batch_idx = 1
    for b_data in batches:
        torch.cuda.empty_cache()  # make sure the cache is emptied to begin the nexxt batch
        b_train = torch.from_numpy(b_data[:,0,:].astype("int64")).squeeze().to(device).long()
        b_upos = torch.from_numpy(b_data[:,1,:].astype("int64")).squeeze().to(device).long()
#         b_deprel = torch.from_numpy(b_data[:,2,:].astype("int64")).squeeze().to(device).long()
#         tensor_data = torch.from_numpy(bdata).to(device).long()  #.double()  #.float()
        
        optimizer.zero_grad()
        txtcode, positions, latent, dec = model(b_train)
        last_latent = latent[-1]
        upos, deprel = dec
#         print(emb.shape,emb.dtype, res.shape, res.dtype)
#         print(upos.shape, b_upos.shape)
#         loss = loss_function(upos, deprel, upos_emb(b_upos), deprel_emb(b_deprel))
#         loss = loss_function(upos, deprel, b_upos, b_deprel)
        # Untill I make it work, work only with the UPOS PoS as it will be faster MUCH faster
#         loss = F.kl_div(upos, upos_emb(b_upos), reduction="batchmean")
        loss = F.nll_loss(upos.view([-1,18]),b_upos.view([-1]))
#         loss = F.cross_entropy(upos.view([-1,18]),b_upos.view([-1]))
#         loss = F.cross_entropy(upos,b_upos)
#         loss = F.mse_loss(upos, upos_emb(b_upos))
        
        loss.backward()
        train_loss += loss.data.item()  # [0]
        writer.add_scalar("Loss/train", loss.data.item(), global_step=epoch*batch_idx)
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Timestamp {} Train Epoch: {} [{}/{} ]\tLoss: {:.6f}'.format(
                datetime.now(),
                epoch, batch_idx , (ndatapoints//len(b_data)),
                loss.data.item() / b_data.shape[0]))
#             batch_loss.append(loss)
        batch_idx += 1
        del(b_train)
        del(b_upos)
#         del(b_deprel)
        torch.cuda.empty_cache()
    writer.add_scalar("EpochLoss/train", train_loss / batch_idx, epoch)
    print('====> Timestamp {} Epoch: {} Average loss: {:.8f}'.format(datetime.now(), epoch, train_loss / ndatapoints))
    return train_loss

In [ ]:
# load testing data ALL the training data
base_dir = "/home/leo/projects/Datasets/text/UniversalDependencies/ud-treebanks-v2.5"
# get all file paths for testing
all_fnames = get_all_files_recurse(base_dir)
fnames = [f for f in all_fnames if "test-charse" in f and f.endswith(".npy")]


In [ ]:
len(fnames)

In [ ]:
# load all test files 
test_data = []
for f in fnames:
    data = np.load(f)
    lang_name = path_leaf(f).split("-ud")[0]
    test_data.append((lang_name, data))

In [ ]:
def test(model, loss_function, test_data, epoch, device, max_data=100):
    model.eval()
    test_loss = 0
    for lang, d in test_data:
        torch.cuda.empty_cache()  # make sure the cache is emptied to begin the nexxt batch
        b_test = torch.from_numpy(d[:max_data,0,:].astype("int64")).squeeze().to(device).long()
        b_upos = torch.from_numpy(d[:max_data,1,:].astype("int64")).squeeze().to(device).long()
#         b_deprel = torch.from_numpy(d[:,2,:].astype("int64")).squeeze().to(device).long()
        _, _, _, dec = model(b_test)
#         last_latent = latent[-1]
        upos, _ = dec
        loss = loss_function(upos.view([-1,18]),b_upos.view([-1]))
#         loss =  loss_function(res, tensor_data).data.item()  # [0]
        test_loss += loss.data.item()
        writer.add_scalar("LangLoss/test/"+lang, loss.data.item(), global_step=epoch)
        del(b_test)
        del(b_upos)
        torch.cuda.empty_cache()
    test_loss /= len(test_data)  # although this is not faire as different languages give different results
    writer.add_scalar("EpochLangLoss/test/", test_loss, global_step=epoch)
    print('epoch: {}====> Test set loss: {:.8f}'.format(epoch, test_loss))

In [ ]:
# reload model from saved state:
# net.network.load_model("./trained_models/conv1dcol", "conv1dcol_kl-div+1000batches-mse-loss_epoch-3_001")

In [ ]:
model = net.to(device)

In [ ]:
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-6, weight_decay=0, amsgrad=False )
# optimizer = torch.optim.AdamW(model.parameters())
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

In [ ]:
loss_function = F.nll_loss

In [ ]:
data_train.shape, data_train.shape[0]//50

In [ ]:
epoch_size = 10000
batch_size = 50
# data = data_train[-1000*batch_size:,:,:]  # just for the trials, use the last 1000 batches only
data = data_train

In [ ]:
print(data.shape)

In [ ]:
epochs = chunks(data, epoch_size, dim=0)
# batches = chunks(data, batch_size, dim=0)

In [ ]:
# %%time
# epoch_count = 0
# test(model, loss_function, test_data, epoch_count, device, max_data=50)

In [ ]:
%%time
epoch_count = 1
for e in epochs:
    batches = chunks(e, batch_size, dim=0)
    eloss = train(model, optimizer, loss_function, batches, epoch_count, epoch_size, device, log_interval=10)
    test(model, loss_function, test_data, epoch_count, device, max_data=50)
    epoch_count+=1


In [ ]:
# %%time
# epoch_count = 2
# eloss = train(model, optimizer, loss_function, batches, epoch_count, len(data), device, log_interval=20)


In [ ]:
model.network.save_model("./trained_models/conv1dcol", "conv1dcol_nll-loss_pretrained_epoch-{}".format(epoch_count))

In [ ]:
epoch_count

Tried different Nx50 sizes for batches but the only one that works is 50, it seems will be the maximum number of samples in each batch for the training in my GPU

The issue is that training does not seem to work correctly.

All training losses (kl_div, mse_loss) seem to learn well only the first 100 batches and then nothing, it oscilates. After several different initializations with kl_div it worked better (the first loss was about initialized to -1 ... ) so initialization seems to take an important role here.

I need to write a test function now to be able to measure with the test datasets and see the real accuracy



In [ ]:
torch.cuda.memory_allocated()

In [ ]:
torch.cuda.memory_cached()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
torch.cuda.memory_allocated()

In [ ]:
torch.cuda.memory_cached()